# Bars Multiplicity determination with Scikit-learn classifiers

Here we test out scikit-learn classification models for multiplicity reconstruction.

In [1]:
label = "nPH"
mode = "barstri"
nmax = 4
dp = 12

In [2]:
import copy
import math
import multiprocessing
import sys
import time

import numpy as np
import pandas as pd
import sklearn
from joblib import Parallel, delayed
from sklearn import *
from sklearn.experimental import enable_hist_gradient_boosting

sys.path.append("..")
from helpers import filename_for, with_timeout

Load ALL the classification models from scikit-learn.
Note that some models are very slow to train with large datasets or crash outright, so we give them a reduced number of (shuffled) rows to learn.
Note that `n_jobs=1` is used, as parallelism is introduced later.

In [3]:
models_a1 = [
    ("BaggingClassifier", sklearn.ensemble.BaggingClassifier(n_jobs=1), 25000),
    ("BernoulliNB", sklearn.naive_bayes.BernoulliNB(), 600000),
    ("CalibratedClassifierCV", sklearn.calibration.CalibratedClassifierCV(cv=5), 10000),  # Slow with some scalers
    ("ComplementNB", sklearn.naive_bayes.ComplementNB(), 600000),
    ("GaussianNB", sklearn.naive_bayes.GaussianNB(), 600000),
]

models_a2 = [
    ("LinearDiscriminantAnalysis", sklearn.discriminant_analysis.LinearDiscriminantAnalysis(), 150000),
    ("LinearSVC", sklearn.svm.LinearSVC(max_iter=20000), 60000),  # slow with unscaled data
    (
        "LogisticRegression",
        sklearn.linear_model.LogisticRegression(solver="lbfgs", multi_class="auto", max_iter=20000),
        5000,
    ),  # slow with unscaled data
    (
        "LogisticRegressionCV",
        sklearn.linear_model.LogisticRegressionCV(cv=5, solver="lbfgs", multi_class="auto", max_iter=20000),
        5000,
    ),
    ("MLPClassifier", sklearn.neural_network.MLPClassifier(), 25000),
    ("MultinomialNB", sklearn.naive_bayes.MultinomialNB(), 600000),
]

models_b = [
    ("NearestCentroid", sklearn.neighbors.NearestCentroid(), 600000),
    (
        "PassiveAggressiveClassifier",
        sklearn.linear_model.PassiveAggressiveClassifier(max_iter=1000, tol=1e-3, n_jobs=1),
        600000,
    ),
    ("Perceptron", sklearn.linear_model.Perceptron(n_jobs=1), 600000),
    ("QuadraticDiscriminantAnalysis", sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis(), 50000),
    ("RidgeClassifier", sklearn.linear_model.RidgeClassifier(), 600000),
    ("RidgeClassifierCV", sklearn.linear_model.RidgeClassifierCV(), 200000),
    ("SGDClassifier", sklearn.linear_model.SGDClassifier(max_iter=5000, tol=1e-3, n_jobs=1), 25000),
]

# Run these sequential due to "buffer source array is read-only" with LokyBackend
models_s = [
    ("AdaBoostClassifier", sklearn.ensemble.AdaBoostClassifier(), 100000),
    ("DecisionTreeClassifier", sklearn.tree.DecisionTreeClassifier(), 600000),
    ("ExtraTreeClassifier", sklearn.tree.ExtraTreeClassifier(), 600000),
    ("ExtraTreesClassifier", sklearn.ensemble.ExtraTreesClassifier(n_estimators=100, n_jobs=-1), 600000),
    ("RandomForestClassifier", sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=-1), 600000),
]

# Models that fail alot
models_o = [
    # ('NuSVC', sklearn.svm.NuSVC(), 'fast'),  # nu infeasible
    # ("RadiusNeighborsClassifier", sklearn.neighbors.RadiusNeighborsClassifier(radius=10, n_jobs=-1), MEDI),
    ("GaussianProcessClassifier", sklearn.gaussian_process.GaussianProcessClassifier(), 800),
    # ('GradientBoostingClassifier', sklearn.ensemble.GradientBoostingClassifier(), 'slow'),  # crashes
    # ('HistGradientBoostingClassifier', sklearn.ensemble.HistGradientBoostingClassifier(), 'slow'),  # crashes?
    ("KNeighborsClassifier", sklearn.neighbors.KNeighborsClassifier(n_jobs=10), 600000),
    (
        "LabelPropagation",
        sklearn.semi_supervised.LabelPropagation(),
        600000,
    ),  # requires too much memory to train with larger datasets
    ("LabelSpreading", sklearn.semi_supervised.LabelSpreading(), 600000),  # bit slow
    ("SVC", sklearn.svm.SVC(gamma="scale"), 20000),  # slow, not timeouted
]

Some models only work with properly scaled data, so we prepare ALL available scalers.

In [4]:
class UnscaledScaler(sklearn.base.TransformerMixin, sklearn.base.BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X, copy=None):
        if isinstance(X, pd.DataFrame):
            return X.to_numpy()
        else:
            return X

    def fit_transform(self, X, y=None):
        return self.transform(X, y)


scalers = [
    ("Unscaled data", UnscaledScaler()),
    ("standard scaling", sklearn.preprocessing.StandardScaler()),
    ("min-max scaling", sklearn.preprocessing.MinMaxScaler()),
    ("max-abs scaling", sklearn.preprocessing.MaxAbsScaler()),
    ("robust scaling", sklearn.preprocessing.RobustScaler(quantile_range=(25, 75))),
    # ("power transformation (Yeo-Johnson)", sklearn.preprocessing.PowerTransformer(method="yeo-johnson")), # complains about shapes
    # ('power transformation (Box-Cox)', sklearn.preprocessing.PowerTransformer(method='box-cox')), # 'strictly zero' meh.
    ("quantile transformation (gaussian pdf)", sklearn.preprocessing.QuantileTransformer(output_distribution="normal")),
    ("quantile transformation (uniform pdf)", sklearn.preprocessing.QuantileTransformer(output_distribution="uniform")),
    ("sample-wise L2 normalizing", sklearn.preprocessing.Normalizer()),
]

## Data preprocessing

Load

In [5]:
files = [
    filename_for(15, dp, 600, 500, n, "inclxx", s, "bars.parquet") for n in range(1, nmax + 1) for s in range(5)
]  # 20)]
dfs = [pd.read_parquet(file) for file in files]
data = pd.concat(dfs, ignore_index=True).sample(frac=1)
data.loc[data["nHits"] == 0, ["nPN", "nPP", "nPH"]] = 0
display(data)

,nPN,nPP,nPH,nHits,nClus,Edep,0,1,2,3,...,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399
127636,3,3,2,17,14,228,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65128,2,2,1,13,6,204,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91389,0,0,0,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120922,3,2,2,6,2,179,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161629,4,2,2,36,5,504,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89847,2,2,1,10,7,122,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144268,3,2,2,33,11,553,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83886,2,2,2,21,8,378,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8268,1,1,1,17,5,346,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Train-Test-Split

In [6]:
msk = np.random.rand(data.shape[0]) < 0.8
traindata = data[msk]
testdata = data[~msk]

print(traindata.shape)
print(testdata.shape)

(159797, 2406)
(40203, 2406)


Scale Features: nHits, nClus, and Edep normally, but HitE / HitT from all bars together

In [7]:
cols_tri = ["nHits", "nClus", "Edep"]
cols_e = [str(i) for i in range(0, dp * 100 * 2, 2)]
cols_t = [str(i + 1) for i in range(0, dp * 100 * 2, 2)]

scalers_trained = [
    (
        sname,
        copy.copy(scaler).fit(traindata[cols_tri]),
        copy.copy(scaler).fit(traindata[cols_e].values.reshape(-1, 1)),
        copy.copy(scaler).fit(traindata[cols_t].values.reshape(-1, 1)),
    )
    for sname, scaler in scalers
]

In [8]:
if mode == "bars":
    data_scaled = [
        (
            sname,
            np.concatenate(
                (
                    # s_tri.transform(traindata[cols_tri]),
                    s_e.transform(traindata[cols_e].values.reshape(-1, 1)).reshape(-1, len(cols_e)),
                    s_t.transform(traindata[cols_t].values.reshape(-1, 1)).reshape(-1, len(cols_t)),
                ),
                axis=1,
            ),
            np.concatenate(
                (
                    # s_tri.transform(testdata[cols_tri]),
                    s_e.transform(testdata[cols_e].values.reshape(-1, 1)).reshape(-1, len(cols_e)),
                    s_t.transform(testdata[cols_t].values.reshape(-1, 1)).reshape(-1, len(cols_t)),
                ),
                axis=1,
            ),
        )
        for sname, s_tri, s_e, s_t in scalers_trained
    ]
elif mode == "barstri":
    data_scaled = [
        (
            sname,
            np.concatenate(
                (
                    s_tri.transform(traindata[cols_tri]),
                    s_e.transform(traindata[cols_e].values.reshape(-1, 1)).reshape(-1, len(cols_e)),
                    s_t.transform(traindata[cols_t].values.reshape(-1, 1)).reshape(-1, len(cols_t)),
                ),
                axis=1,
            ),
            np.concatenate(
                (
                    s_tri.transform(testdata[cols_tri]),
                    s_e.transform(testdata[cols_e].values.reshape(-1, 1)).reshape(-1, len(cols_e)),
                    s_t.transform(testdata[cols_t].values.reshape(-1, 1)).reshape(-1, len(cols_t)),
                ),
                axis=1,
            ),
        )
        for sname, s_tri, s_e, s_t in scalers_trained
    ]

## Run all model/scaler combinations, in parallel.
Note that we use timeouts per task, as setting at timeout in joblib will throw everything.

In [9]:
@with_timeout(1200 * 3)
def train_model(mname, modelorg, speed, sname, x_train, x_test):
    # These get killed without error?
    if mname == "RadiusNeighborsClassifier" and sname != "Unscaled data":
        return (mname, sname, np.NaN, speed, np.NaN, "Skipped")
    try:
        model = sklearn.base.clone(modelorg)
        start = time.time()
        model.fit(x_train[0:speed], traindata.head(speed)[[label]].values.ravel())
        end = time.time()

        y_pred = model.predict(x_test)
        y_true = testdata[[label]].values.ravel()

        bac = sklearn.metrics.balanced_accuracy_score(y_true, y_pred)
        return (mname, sname, bac, speed, (end - start), "ok")
    except Exception as err:
        return (mname, sname, np.NaN, speed, np.NaN, err)


def train_model_wrap(mname, modelorg, speed, sname, x_train, x_test):
    ret = train_model(mname, modelorg, speed, sname, x_train, x_test)
    if ret:
        return ret
    else:
        return (mname, sname, np.NaN, speed, np.NaN, "Timeout")

In [10]:
try:
    results_a1 = Parallel(n_jobs=8, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in data_scaled
        for mname, modelorg, speed in models_a1
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  40 out of  40 | elapsed: 14.8min finished


In [11]:
try:
    results_a2 = Parallel(n_jobs=8, verbose=3)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in data_scaled
        for mname, modelorg, speed in models_a2
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed: 17.7min
[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed: 113.1min finished


In [12]:
try:
    results_b = Parallel(n_jobs=1, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in data_scaled
        for mname, modelorg, speed in models_b
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:741: RuntimeWarning: overflow encountered in square
  norm2.append(np.sum(X2 ** 2, axis=1))
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.31187e-24): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variable

In [13]:
try:
    results_s = Parallel(n_jobs=1, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in data_scaled
        for mname, modelorg, speed in models_s
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 56.8min finished


In [14]:
results_o = []
for mname, modelorg, speed in models_o:
    try:
        tmp = Parallel(n_jobs=10, verbose=1)(
            delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
            for sname, x_train, x_test in data_scaled
        )
        results_o.extend(tmp)
    except Exception as err:
        print(err)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 out of   8 | elapsed:  6.4min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 out of   8 | elapsed: 60.3min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 out of   8 | elapsed: 38.9min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 out of   8 | elapsed: 28.5min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 out of   8 | elapsed: 56.8min finished


In [15]:
results = results_a1 + results_a2 + results_b + results_s + results_o
resultsdf = pd.DataFrame(results)
pd.options.display.max_rows = 999
resultsdf.columns = ["Model", "Scaler", "BAC", "Events", "Time", "Status"]

max_events = traindata.shape[0]
resultsdf.loc[resultsdf["Events"] > max_events, "Events"] = max_events
resultsdf["Speed"] = resultsdf["Events"] / resultsdf["Time"]
resultsdf["OptEvents"] = resultsdf["Speed"] * 1200
resultsdf["OptEvents"] = resultsdf["OptEvents"].apply(lambda x: (0 if math.isnan(x) else round(x, 0)))
resultsdf["OptEvents"] = resultsdf["OptEvents"].astype("int")
resultsdf.loc[resultsdf["OptEvents"] > max_events, "OptEvents"] = max_events

resultsdf.sort_values(by=["BAC", "Time"], ascending=[False, True], inplace=True)
# resultsdf.sort_values(by=["Model", "Scaler"], ascending=[True, True], inplace=True)
resultsdf.style.hide_index().format({"BAC": "{:.2%}", "Time": "{:.2f}"}).bar(subset=["BAC"], color="lightgreen").bar(
    subset=["Time"], color="lightblue"
)

Model,Scaler,BAC,Events,Time,Status,Speed,OptEvents
LinearDiscriminantAnalysis,quantile transformation (gaussian pdf),69.18%,150000,231.05,ok,649.204395,159797
MLPClassifier,quantile transformation (gaussian pdf),69.04%,25000,329.64,ok,75.839592,91008
LinearDiscriminantAnalysis,quantile transformation (uniform pdf),68.03%,150000,283.75,ok,528.629796,159797
LinearDiscriminantAnalysis,sample-wise L2 normalizing,67.05%,150000,263.94,ok,568.311052,159797
SVC,min-max scaling,67.01%,20000,716.21,ok,27.924628,33510
SVC,max-abs scaling,67.01%,20000,838.54,ok,23.851121,28621
LogisticRegression,max-abs scaling,65.73%,5000,14.72,ok,339.604215,159797
LogisticRegression,min-max scaling,65.73%,5000,14.39,ok,347.565614,159797
LogisticRegressionCV,min-max scaling,65.73%,5000,1086.87,ok,4.600361,5520
LogisticRegressionCV,max-abs scaling,65.72%,5000,1089.98,ok,4.587253,5505
